Two sets of data are created here:
1. Filtered: The raw data is filtered by applying a low-pass filter(cutoff frequency of 45 Hz), a high-pass filter(cutoff frequency of 1 Hz) and a notch filter(cutoff frequency of 50 Hz).
2. Unfiltered: The raw data is filtered by applying a high-pass filter(cutoff frequency of 1 Hz) and a notch filter(cutoff frequency of 50 Hz).

Both os sets of data are then rereferenced to the averaged EEG data.

Resulting data is saved as fif files, and pickled as a dataframe

In [1]:
#import 
import mne
import os
import pandas as pd
from mne.utils import ProgressBar
from mne import set_log_level
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
#matplotlib.use('Qt5Agg')
import pickle

Filter data from fif files

In [2]:
files = [ f.path for f in os.scandir('dataset2/data/mne') if f.is_file()]

In [ ]:
n_files = len(files)
data_filtered = pd.DataFrame()

# Make filtered dataset
print("Filtering {n} files".format(n=n_files))
# Initialize a progress bar.
progress = ProgressBar(n_files, mesg='Filtering')
for name in files:
    raw = mne.io.read_raw_fif(name, preload=True, verbose='error')
    # high pass filter above 1
    raw.filter(l_freq=1, h_freq=None, phase='zero',
                fir_window='hamming', l_trans_bandwidth='auto',
                h_trans_bandwidth='auto', filter_length='auto')
    #low pass filter before 45
    raw.filter(l_freq=None, h_freq=45, phase='zero',
                fir_window='hamming', l_trans_bandwidth='auto',
                h_trans_bandwidth='auto', filter_length='auto')
    #notch filter at 50 Hz
    raw.notch_filter(np.arange(50, 95, 50), fir_design='firwin')
    #set eeg reference
    raw.set_eeg_reference('average', projection=False)  

    #save filtered data 
    if not os.path.exists(f'dataset2/data/filtered-ref'):
        os.makedirs(f'dataset2/data/filtered-ref')
    raw.save(f"dataset2/data/filtered-ref/{os.path.basename(name)}_filtered.fif")
    plot_raw = raw.plot_psd()
    plot_raw.savefig(f"dataset2/data/filtered-ref/{os.path.basename(name)}_filtered.png")
    
    temp_df = pd.DataFrame(raw._data.T, columns=raw.ch_names)
    temp_df['subject'] = f'{os.path.basename(name)}'
    temp_df['time'] = raw.times
    data_filtered = data_filtered.append(temp_df, ignore_index=True)
    progress.update_with_increment_value(1)


#pickle dataframe
with open('dataset2/data/filtered_ref.pkl', 'wb') as f:
    pickle.dump(data_filtered, f)


Make unfiltered dataset

In [ ]:
n_files = len(files)
data_unfiltered = pd.DataFrame()

# Make unfiltered dataset
print("Filtering(basic) {n} files".format(n=n_files))
# Initialize a progress bar.
progress = ProgressBar(n_files, mesg='Filtering')
for name in files:
    raw = mne.io.read_raw_fif(name, preload=True, verbose='error')
    # high pass filter above 0.5
    raw.filter(l_freq=1, h_freq=None, phase='zero',
                fir_window='hamming', l_trans_bandwidth='auto',
                h_trans_bandwidth='auto', filter_length='auto', verbose='error')
    #notch filter at 50 Hz
    raw.notch_filter(np.arange(50, 95, 50), fir_design='firwin', verbose='error')
    #set eeg reference
    raw.set_eeg_reference('average', projection=False)  
    if not os.path.exists(f'dataset2/data/unfiltered-ref'):
        os.makedirs(f'dataset2/data/unfiltered-ref')
    #save data
    raw.save(f"dataset2/data/unfiltered-ref/{os.path.basename(name)}_unfiltered.fif", verbose='error')
    plot_raw = raw.plot_psd()
    plot_raw.savefig(f"dataset2/data/unfiltered-ref/{os.path.basename(name)}_unfiltered.png")

    temp_df = pd.DataFrame(raw._data.T, columns=raw.ch_names)
    temp_df['subject'] = f'{os.path.basename(name)}'
    temp_df['time'] = raw.times
    data_unfiltered = data_unfiltered.append(temp_df, ignore_index=True)
    progress.update_with_increment_value(1)

#pickle dataframe
with open('dataset2/data/unfiltered_ref.pkl', 'wb') as f:
    pickle.dump(data_unfiltered, f)